In [ ]:
# ==============================================================================
# CELL 1: BOOTSTRAP - INTELLIGENT ENVIRONMENT SETUP
# ==============================================================================
# This cell checks if the required dependencies are already installed. If not,
# it downloads and runs a setup script to prepare the Colab environment.

import sys
import subprocess
import os
import shutil

# --- Configuration ---
# Set to True to install the full OpenStudio CLI (required for running Measures)
INSTALL_CLI = False
# Set the branch of the toolkit to use ('main' for stable, 'dev' for development)
TOOLKIT_BRANCH = 'main'

# --- Verification and Execution Logic ---
try:
    # Attempt to import the core libraries.
    import openstudio
    import openstudio_toolkit
    
    # If INSTALL_CLI is True, also check for the command line interface.
    if INSTALL_CLI and not shutil.which('openstudio'):
        # This will trigger the installation if the CLI is missing but the library is present.
        raise ImportError("OpenStudio CLI not found")

    print("✅ Environment is already set up. Skipping installation.")

except ImportError:
    print("⏳ Required dependencies not found. Proceeding with setup...")
    
    bootstrap_script = 'bootstrap.py'
    # Download the bootstrap script if it doesn't exist OR if it's an empty file.
    if not os.path.exists(bootstrap_script) or os.path.getsize(bootstrap_script) == 0:
        print("--> Downloading bootstrap script...")
        bootstrap_url = f"https://raw.githubusercontent.com/roruizf/OpenStudio-Toolkit/{TOOLKIT_BRANCH}/{bootstrap_script}"
        os.system(f"wget -q -O {bootstrap_script} {bootstrap_url}")

    # Final check to ensure the downloaded script is not empty before running
    if os.path.exists(bootstrap_script) and os.path.getsize(bootstrap_script) > 0:
        print("--> Running environment setup...")
        cli_flag = "--install-cli" if INSTALL_CLI else ""
        os.system(f"python {bootstrap_script} --branch {TOOLKIT_BRANCH} {cli_flag}")
        
        print("\n💥 KERNEL RESTART is likely required for the changes to take effect.")
        print("Please go to 'Runtime' > 'Restart session' in the menu, then run the next cell.")
    else:
        print("❌ ERROR: Failed to download the bootstrap script. Please check the URL and your connection.")

In [ ]:
# ==============================================================================
# CELL 2: RUN TASK
# ==============================================================================
# IMPORTANT: Run this cell AFTER restarting the kernel from Cell 1 if needed.

import os
import pandas as pd
from datetime import datetime, timedelta

# --- Imports from the Toolkit ---
from openstudio_toolkit.tasks.results_analysis import get_timeseries_from_sql
from openstudio_toolkit.tasks.results_analysis.get_timeseries_from_sql import Variable

# --- User Inputs ---
# Define the path to your simulation's SQL output file
# For local execution, you might have a path like this:
LOCAL_SQL_PATH = '../../examples/eplusout.sql' 

# Define the variables you want to extract
# Variable = namedtuple("Variable", "key type units")
VARIABLES_TO_EXTRACT = [
    Variable(key=None, type="Site Outdoor Air Drybulb Temperature", units="C"),
    Variable(key=None, type="Zone Air Temperature", units="C")
]
# Define the reporting frequency
REPORTING_FREQUENCY = 'hourly' # Options: "timestep", "hourly", "daily", "monthly", "runperiod", "annual"

# Set to None to ignore, or provide a datetime object.
START_DATE = datetime(2006, 7, 1, 0, 0, 0)  # Example: Start on July 1st
END_DATE = datetime(2006, 7, 7, 0, 0, 0)    # Example: End on July 7th

# --- Main Execution Block ---
# For this task, we assume the SQL file is always local, not in GDrive
sql_file_path = os.path.abspath(LOCAL_SQL_PATH)

if os.path.exists(sql_file_path):
    report = get_timeseries_from_sql.validator(sql_path=sql_file_path)
    print("--- Validation Report ---")
    for msg in report['messages']: print(msg)
    
    if report['status'] == 'READY':
        results_df = get_timeseries_from_sql.run(
            sql_path=sql_file_path,
            variables=VARIABLES_TO_EXTRACT,
            frequency=REPORTING_FREQUENCY,
            start_date=START_DATE,
            end_date=END_DATE
        )
        
        if results_df is not None:
            print("\n--- Verifying Results ---")
            print(f"Successfully extracted data from {results_df.index.min()} to {results_df.index.max()}:")
            display(results_df.head())
            display(results_df.tail())
else:
    print(f"❌ ERROR: SQL file not found at the specified path: {sql_file_path}")